# Парсинг вакансий hh.ru

In [1]:
import json
import re

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm

В переменной `REQUEST` храним поисковый запрос. Например `data scientist`.

In [2]:
REQUEST = 'data scientist'

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                         'AppleWebKit/537.36 (KHTML, like Gecko)' 
                         'Chrome/79.0.3945.79 Safari/537.36'}

URL = 'https://hh.ru/search/vacancy'

PARAMS = {
    "area": "1", 
    "L_is_autosearch": "false", 
    "clusters": "true", 
    "enable_snippets": "true", 
    "text": REQUEST
}

Функция `getNumberOfPages` возвращает номер последней страницы или другими словами их количество.

In [3]:
def getNumberOfPages():
    request = requests.get(URL, params=PARAMS, headers=HEADERS)
    parsed_html = bs(request.text, 'lxml')
    page_numbers = parsed_html.findAll('a', {'data-qa': 'pager-page'})
    last_page_number = int(str(page_numbers[-1].text)) if page_numbers else 0
    
    return last_page_number

In [4]:
getNumberOfPages()

8

Функция парсинга одной отдельной страницы с вакансиями.

In [5]:
def parseHHPage(data, page_num=0):  
    params = PARAMS
    params.update(page=str(page_num))
    request = requests.get(URL, params=params, headers=HEADERS)
    
    parsed_html = bs(request.text, 'lxml')
    
    vacancy_list = parsed_html.findAll('div', {'class': 'vacancy-serp-item'})
    
    for vacancy in vacancy_list:
        d = {}
        title = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
        d['Title'] = title.text
        
        salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        d['Salary from'], d['Salary to'], d['Currency'] = parseSalary(salary)
        
        d['Link'] = re.sub(r'\?.*$', '', title['href'])
        
        location = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'})
        d['Location'] = location.text if location else None
        
        company = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
        d['Company'] = company.text if company else None
        
        data.append(d)
        
    return parsed_html

Функция парсинга зарплаты. Зарплата хранится текстом, поэтому будем её парсить.

In [6]:
def parseSalary(salary):
    sal_from = None
    sal_to = None
    currency = None 
    if salary:
        salary = re.sub(r'[^\w\-–]', '', salary.text, re.UNICODE).replace('руб', 'RUR')
        if not salary.find('от'):
            re_search = re.search(r'(\d+)(.+)', salary)
            if re_search:
                sal_from = int(re_search.group(1))
                currency = re_search.group(2)
        elif not salary.find('до'):
            re_search = re.search(r'(\d+)(.+)', salary)
            if re_search:
                sal_to = int(re_search.group(1))
                currency = re_search.group(2)
        else:
            re_search = re.search(r'(\d+)[^d](\d+)(.+)', salary)
            if re_search:
                sal_from = int(re_search.group(1))
                sal_to = int(re_search.group(2))
                currency = re_search.group(3)
    return sal_from,  sal_to, currency

Запускаем парсинг по всем страницам результата поиска

In [7]:
data = []
last_page = getNumberOfPages()
for i in tqdm(range(1, last_page + 1)):
    parseHHPage(data, i)

  0%|          | 0/8 [00:00<?, ?it/s]

Смотрим результат. Чтобы не выводить все строки, оставим только те, в которых указана зарплата. Хотя бы одна из границ.

In [8]:
df = pd.DataFrame(data)
df[df['Salary from'].notnull() | df['Salary to'].notnull()]

,Title,Salary from,Salary to,Currency,Link,Location,Company
7,Data Scientist,NaN,350000.0,RUR,https://hh.ru/vacancy/50108323,"Москва, Киевская",3LOGIC GROUP
16,Data Engineer,50000.0,500000.0,RUR,https://hh.ru/vacancy/50651699,Москва,RockIT studio
21,Data Scientist,220000.0,NaN,RUR,https://hh.ru/vacancy/48484869,Москва,Borzo
25,Data scientist (computer vision),200000.0,NaN,RUR,https://hh.ru/vacancy/50571106,Москва,АО Гознак
34,Middle Data Engineer,180000.0,250000.0,RUR,https://hh.ru/vacancy/50048134,Москва,Educate Online Inc
38,Data Scientist,200000.0,300000.0,RUR,https://hh.ru/vacancy/50579318,Москва,SegmentStream
52,Data Scientist,NaN,350000.0,RUR,https://hh.ru/vacancy/42011548,Москва,ООО РАБЛЗ
75,Data Scientist,2000.0,8000.0,USD,https://hh.ru/vacancy/50337537,Москва,Goltsblat Capital
78,Senior Data Scientist,NaN,350000.0,RUR,https://hh.ru/vacancy/49462557,Москва,Spice Agency
83,Data Analyst,100000.0,250000.0,RUR,https://hh.ru/vacancy/48648580,Москва,ООО РАБЛЗ


Можно сохранить json-файл.

In [9]:
with open('vacancies.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)